<a href="https://colab.research.google.com/github/ujjwalrajput31/eda/blob/main/Book_Recommender_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
0import numpy as np
import pandas as pd


In [ ]:
books=pd.read_csv('books.csv',sep=';',on_bad_lines='skip',encoding='latin-1', engine='python')

FileNotFoundError: [Errno 2] No such file or directory: 'books.csv'

In [ ]:
books.head()

In [ ]:
books.columns


In [ ]:
books=books[['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher']].copy()

In [ ]:
books.head()

In [ ]:
books.rename(columns={'Book-Title':'title','Book-Author':'author','Year-Of-Publication':'year','Publisher':'publisher'},inplace=True)

In [ ]:

books.head()

In [ ]:
user=pd.read_csv('users.csv',sep=";",on_bad_lines='skip',encoding='latin-1', engine='python')

In [ ]:
user.head(2)

In [ ]:
user.rename(columns={'User-ID':'user_id','Location':'location','Age':'age'},inplace=True)

In [ ]:
user.head(2)

In [ ]:
ratings=pd.read_csv('ratings.csv',sep=';',encoding='latin=1',on_bad_lines='skip',engine='python')

In [ ]:
ratings.head(2)

In [ ]:
ratings.rename(columns={'User-ID':'user_id','Book-Rating':'rating'},inplace=True)

In [ ]:
ratings.rename(columns={'isbn':'ISBN'},inplace=True)
ratings.head(2)

In [ ]:
books.shape

In [ ]:
user.shape

In [ ]:
ratings.shape

In [ ]:
ratings['user_id'].value_counts().shape

In [ ]:
x=ratings['user_id'].value_counts()>200

In [ ]:
x[x].shape

In [ ]:
y=x[x].index

In [ ]:
y

In [ ]:
ratings=ratings[ratings['user_id'].isin(y)]

In [ ]:
ratings.shape

In [ ]:
ratings_with_books=ratings.merge(books,on='ISBN')

In [ ]:
ratings_with_books

In [ ]:
ratings_with_books.shape

In [ ]:
number_rating=ratings_with_books.groupby('title')['rating'].count().reset_index()

In [ ]:
number_rating.rename(columns={'rating':'num_of_ratings'},inplace=True)

In [ ]:
number_rating.head()

In [ ]:
final_rating=ratings_with_books.merge(number_rating,on='title')

In [ ]:
final_rating

In [ ]:
final_rating=final_rating[final_rating['num_of_ratings']>=50]

In [ ]:
final_rating.shape

In [ ]:
final_rating.drop_duplicates(['user_id','title'],inplace=True)

In [ ]:
final_rating.shape

In [ ]:
books_pivot=final_rating.pivot_table(columns='user_id',index='title',values='rating')

In [ ]:
books_pivot.shape

In [ ]:
books_pivot.fillna(0,inplace=True)

In [ ]:
books_pivot

In [ ]:
from scipy.sparse import csr_matrix
book_sparse=csr_matrix(books_pivot)

In [ ]:
type(book_sparse)

In [ ]:
from sklearn.neighbors import NearestNeighbors
model=NearestNeighbors(algorithm='brute')

In [ ]:
model.fit(book_sparse)

In [ ]:
distances,suggestions=model.kneighbors(books_pivot.iloc[237,:].values.reshape(1,-1),n_neighbors=6)

In [ ]:
distances

In [ ]:
suggestions

In [ ]:
for i in range(len(suggestions)):
  print(books_pivot.index[suggestions[i]])

In [ ]:
np.where(books_pivot.index=='Animal Farm')[0][0]

In [ ]:
def recommend_book(book_name):
  book_id=np.where(books_pivot.index==book_name)[0][0]
  distances,suggestions=model.kneighbors(books_pivot.iloc[book_id,:].values.reshape(1,-1),n_neighbors=6)

  for i in range(len(suggestions)):
    if i==0:
      print("The suggestions for",book_name,"are:")
    if not i:
      print(books_pivot.index[suggestions[i]])

In [ ]:
recommend_book('Animal Farm')